# INTRODUCTION

Using SQL we will be answers a series of questions using the Chinook database. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## SECTION 1: Overview of Data

Looking at the master database, we see that there are 11 different tables that are interconnected to one another which contains the following information:

| table name        | description                       |
|-------------------|-----------------------------------|
| album             | album purchased                   |
| artist            | the artist                        |
| customer          | the customer                      |
| employee          | the employee (may also be linked) |
| genre             | genre of music                    |
| invoice           | invoice for each customer         |
| invoice_line      | invoice line for each purchase    |
| media_type        | media type used for the song      |
| playlist          | the playlist                      |
| playlist_track    | tracks on playlist                |
| track             | the song track                    |


In [3]:
%%sql
SELECT *
FROM album
Limit 3;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2


In [4]:
%%sql
SELECT *
FROM artist
Limit 3;

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith


## Section 2: Selecting Album to Purchase

Working on the premise that the Chinook record store has signed a deal with a new record label, we want to select the first 3 albums to be added to the store from the following list:

| **Artist Name**      | **Genre**   |
|----------------------|-------------|
| Regal                | Hip-Hop     |
| Red Tone             | Punk        |
| Meteor and the Girls | Pop         |
| Slim Jim Bites       | Blues       |
     
Seeing as the record label specializes in **American artists** and that Chinook has received some money to advertise these artists **in the USA**, we are interested in finding out which genres sell best in the US. 

The purpose of this query is to find out **which genre(s) sell the most tracks in the USA**.


**THOUGHT PROCESS:**

1. Need the following tables (in path order) Customer, Invoice, Invoice_line, Track, Genre

2. JOIN ON the following:
   c.customer_id = i.customer_id
   i.invoice_id = il.invoice_id
   il.track_id = t.track_id
   t.genre_id = g.genre_id

3. Customer + Invoice_line + Genre contains the columns that would be needed to create the return query, **HOWEVER** track + invoice are used as linkers (i.e. these tables don't have direct columns needed for the return query). 

4. Since USA is the focus, we can use "WITH" CLAUSE to create a temp subquery to create an isolated table for recruiting only USA customers. 

5. Based on point #4, we can create a subquery that would include the following tables: customer AS "c", invoice AS "i", and invoice_line AS "il" to get columns exclusive to USA only (i.e. g.name = "USA" where g = genre). 

6. To get the total number of tracks sold, we will need to use COUNT(invoice_line_id). This value needs to be specific to a given genre. 

7. To get the percentage, divide the total number of tracks sold per given genre / total number of tracks sold.

In [5]:
%%sql 

WITH usa_only AS
    (SELECT il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id 
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    WHERE c.country = "USA")


SELECT 
        g.name AS "genre", 
        COUNT(uo.invoice_line_id) AS "total_tracks_sold", 
        ROUND(CAST(COUNT(uo.invoice_line_id) AS FLOAT)/
              (SELECT COUNT(*) FROM usa_only) * 100, 2) AS "percentage"
FROM usa_only AS "uo"
INNER JOIN genre AS g ON g.genre_id = t.genre_id
INNER JOIN track AS t ON t.track_id = uo.track_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,total_tracks_sold,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the findings, it appears as though the most popular genres from the given list of new artist based on number of tracks sold are punk (12.4%), blues (3.4%) and pop (2.1%). Considering this, it would be best to recommend the Red Tones, Meteor and the Girls and Slim Jim Bites as the artists that should be pushed to drive record sales. 

It should be noted that these three genres only make up approx. 17.9% of total tracks sold. The Rock genre appears to be the most popular genre and should be focused on. 


## SECTION 3: Analyzing Employee Sales Performance

As each customer for the Chinook store gets assigned a **sales support agent** within the company for their first purchase, our task is to analyse the purchases of customers belonging to **each employee** as a measure of performance. 

The return query that would find the total dollar amount of sales assigned to each sales support agent within the company.  


**THOUGHT PROCESS**:

1) The following tables needed to answer this question include (in order): employee -> customer -> invoice 

2) I would need the following linkages:
    e.employee_id = c.support_rep_id
    c.customer_id = i.customer_id 
    
3) Need a way to isolate only sales support agent, so essentially need a subquery that contains only data for sales support agent. So use a WITH Clause. The names should only include "Jane Peacock", "Margaret Park" and "Steve Johnson". 

4) In generating the main query, we will need the following information: 

         > NAME OF THE SALES SUPPORT AGENT (so first_name || " " || last_name AS "sales_support_agent_name")
         > SUM TOTAL SPENT BY CUSTOMER (so SUM(total) AS "total_sales")

5) It would also be interesting to look at the influence of "hire_date" from the employees table to see if there is a pattern.


In [10]:
%%sql
WITH sales_support AS
(SELECT i.customer_id, c.support_rep_id, SUM(i.total) AS "total_sales"
 FROM invoice AS i
 INNER JOIN customer AS c ON c.customer_id = i.customer_id
 GROUP BY 1, 2)

SELECT 
    e.first_name || " " || e.last_name AS "employee_name",
    e.hire_date,
    SUM(ss.total_sales) AS "Total Sales"
FROM sales_support AS ss
INNER JOIN employee AS e ON e.employee_id = ss.support_rep_id
GROUP BY 1
ORDER BY 3 DESC
;

Done.


employee_name,hire_date,Total Sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


## SECTION 4: Analysing Sales by Country 

The purpose of this section is to analyze the sales data for customers from each different country as to find out information pertaining to 

1) Number of customers 

2) total dollar amount of tracks sold to that given country 

3) the average number of tracks that are sold in a single order to each customer within a given country 

4) the average sales amount for a given customer order within each given country. 

However, it is important to note that for some countries there will be only one customer, so we need to include a way to include these individual country to a special category of country called "other" which will be included into the bottom of the table. 

In [95]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [92]:
%%sql

WITH 
    country_count AS
        (SELECT
            country,        
            (CASE
                 WHEN COUNT(country) = 1 THEN 'Other'
                 ELSE country
             END) as country_or_other,
            COUNT(customer_id) as total_customers_per_country
         FROM customer
         GROUP BY country
        ),
        
    country_sales AS
        (SELECT 
            c.country,
            SUM(i.total) as total_sales,
            COUNT(i.invoice_id) as count_of_sales

        FROM customer as c
        LEFT JOIN invoice as i ON i.customer_id = c.customer_id

        GROUP BY c.country
        ),
    
    sort_table AS
        (SELECT
            country_or_other as country_name_for_sort,
            (CASE
              WHEN country_or_other == 'Other' THEN 1
              ELSE 0
            END) AS sort
         FROM country_count
         WHERE sort = country
        )
    
SELECT
    cc.country_or_other as country_name,
    SUM(cc.total_customers_per_country) as total_customers,
    ROUND(SUM(cs.total_sales), 2) as total_sales,
    ROUND((SUM(cs.total_sales)) / (SUM(cc.total_customers_per_country)), 2) as avg_sales_per_customer,
    ROUND((SUM(cs.total_sales)) / (SUM(cs.count_of_sales)), 2) as avg_order_value
    
FROM country_count as cc
LEFT JOIN country_sales as cs ON cs.country = cc.country
LEFT JOIN sort_table as st ON st.country_name_for_sort = cc.country_or_other

GROUP BY country_name
ORDER BY 3 DESC

Done.


country_name,total_customers,total_sales,avg_sales_per_customer,avg_order_value
Other,15,1094.94,73.0,7.45
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.65,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.56,6.38
India,2,183.15,91.57,8.72



Based on the data, there may be opportunity in the following countries:

Czech Republic
United Kingdom
India
It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Album vs Individual Tracks

In [96]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.